In [1]:
#when saving files from Atmotube sensor to Test sensor data folder always change file name to include YYYY-MM-DD-
#at the beginning of the file name - this allows matching of dates with gpx data

In [2]:
#input file paths
sensor_file_path='/Users/sebastianherbst/Dissertation/Data processing/Test sensor data/2024-05-24_text-8C89FB67B0A2-1.txt'
gpx_file_path='/Users/sebastianherbst/Dissertation/Data processing/Test sensor data/Test gpx data/2024-05-24_triplogger-track-20240524T205818+0100.gpx'

#cache storage
sensor_cache_file_path='/Users/sebastianherbst/Dissertation/Data processing/cache_sensordata_file'

#output file paths
sensor_output_csv_encoded_path='/Users/sebastianherbst/Dissertation/Data processing/Output/2024-05-24_time_encoded_csv_test'
encoded_sensor_and_gpx_path='/Users/sebastianherbst/Dissertation/Data processing/Output/2024-05-24_encoded_sensor_and_gpx.csv'

In [68]:
#imports libraries
import pandas as pd
import regex as re
import numpy as np
import re
import gpxpy
import gpxpy.gpx
from datetime import datetime, timedelta
import os

In [4]:
sep_sens='#' #separator in sensor data

In [5]:
#opens raw data from txt file (data_raw) and creates list (data_list)
data_raw = open(sensor_file_path)
#data_list = data_raw.readlines()
#print(data_list)


In [6]:
#select relevant data from list (data_list) and move to new file (data_write)
#Lines with relevant data in txt file are 126 or 136 elements in length

data_raw_ed = open(sensor_file_path, 'r+')
data_write = open(sensor_cache_file_path, 'w')

l=[]

for i in data_raw_ed.readlines():
    if len(i) >= 126:
        l.append(i)

data_write.write('#'.join(l))
data_raw_ed.close()
data_write.close()


In [7]:
#create string (g_str) file from list containing all lines with relevant data (data_write)
data_write_r = open(sensor_cache_file_path, 'r')
data_write_str = str(data_write_r.read())
sep_sens_str=str(sep_sens) #add separator to beginning of first line
g=[sep_sens_str+data_write_str] 
g_str = str(g)
data_write_r.close()

print(g_str)
#print(g)

['#20:25:08,Connected Device,normal,Discovered Characteristics DB450002-8E9A-4818-ADD7-6ED94A328AB4, DB450003-8E9A-4818-ADD7-6ED94A328AB4, DB450004-8E9A-4818-ADD7-6ED94A328AB4, and DB450005-8E9A-4818-ADD7-6ED94A328AB4 for Service DB450001-8E9A-4818-ADD7-6ED94A328AB4.\n#20:25:08,Connected Device,normal,Discovered Characteristics UART RX Characteristic and UART TX Characteristic for Service Nordic UART Service.\n#20:25:08,Connected Device,normal,Discovered Client Characteristic Configuration for Characteristic DB450002-8E9A-4818-ADD7-6ED94A328AB4\n#20:25:08,Connected Device,normal,Discovered Client Characteristic Configuration for Characteristic DB450003-8E9A-4818-ADD7-6ED94A328AB4\n#20:25:08,Connected Device,normal,Discovered Client Characteristic Configuration for Characteristic DB450004-8E9A-4818-ADD7-6ED94A328AB4\n#20:25:08,Connected Device,normal,Discovered Client Characteristic Configuration for Characteristic DB450005-8E9A-4818-ADD7-6ED94A328AB4\n#20:28:01,Connected Device,normal,

In [8]:
#loading values of g into a pandas dataframe
g_df = pd.DataFrame({'ALL':g})
g_df['ALL'] = g_df['ALL'].str.split('\n')
g_df = g_df.explode('ALL')
g_df = g_df[:-1] #remove last row - additional '\n' left at the end
print(g_df)

                                                  ALL
0   #20:25:08,Connected Device,normal,Discovered C...
0   #20:25:08,Connected Device,normal,Discovered C...
0   #20:25:08,Connected Device,normal,Discovered C...
0   #20:25:08,Connected Device,normal,Discovered C...
0   #20:25:08,Connected Device,normal,Discovered C...
..                                                ...
0   #20:57:34,Connected Device,normal,Updated Valu...
0   #20:57:36,Connected Device,normal,Updated Valu...
0   #20:57:36,Connected Device,normal,Updated Valu...
0   #20:57:38,Connected Device,normal,Updated Valu...
0   #20:57:41,Connected Device,normal,Updated Valu...

[1126 rows x 1 columns]


In [65]:
#filter for only time and measurements in raw data using regex; then import into pandas datafile

#find timestamps and measurements
time = re.findall('(?<=\#)(.{8})',g_str)
values = re.findall('(?<=to\s)(.*?)(?=\.)',g_str)
#All = re.findall('\..*\.',g_str) #doesnt work on this - not every line ends with '.' - instead use g_df data frame
All = g_df['ALL'] #values from g_df data frame - used instead of re.findall(g_str)
#print(time,values)


#produce dataframe and remove lines that do not include relevant data (describe sensor activation)
df1 = pd.DataFrame({'time':time,'All':All})
find_del_line = df1[~df1.All.astype(str).str.contains("Discovered")]
time1 = find_del_line['time'] #update time to match file without irrelevant data

df = pd.DataFrame({'time' : time1, 'values' : values}) #create dataframe (df) showing time and measured air quality (encoded) values

#add date to timestamp in sensor data
date = sensor_file_path[69:79]
df['time'] = date +' '+ df['time'].astype(str)
df['time'] = pd.to_datetime(df['time'], format='ISO8601')

print(date)
print(df)

df.to_csv(sensor_output_csv_encoded_path) #writes results file (for encoded sensor measurements)

2024-05-24
                  time                         values
0  2024-05-24 20:28:01  E201 006B 0200 F102 00FE 0100
0  2024-05-24 20:28:03            3212 088D 0100 D606
0  2024-05-24 20:28:03  DF01 0082 0200 D202 00FA 0100
0  2024-05-24 20:28:05            3312 FE8C 0100 D606
0  2024-05-24 20:28:05  DB01 0062 0200 E002 00F6 0100
..                 ...                            ...
0  2024-05-24 20:57:34  1102 00AD 0200 0203 002F 0200
0  2024-05-24 20:57:36            370E 088D 0100 9605
0  2024-05-24 20:57:36  0702 0097 0200 1403 0025 0200
0  2024-05-24 20:57:38  0002 0091 0200 0603 001E 0200
0  2024-05-24 20:57:41  F801 007F 0200 EB02 0015 0200

[1120 rows x 2 columns]


In [10]:
#use this space here to add location data to measurements

In [72]:
#read gpx data
with open(gpx_file_path, 'r') as f:
    gpx = gpxpy.parse(f)

# Convert to a dataframe one point at a time.
gpx_points = []
for segment in gpx.tracks[0].segments:
    for p in segment.points:
        gpx_points.append({
            'time': p.time,
            'latitude': p.latitude,
            'longitude': p.longitude,
            'elevation': p.elevation,
        })
df_gpx = pd.DataFrame.from_records(gpx_points)

#Remove +00:00 from the end of each timestamp
gpx_time_str = df_gpx['time']
gpx_time_strip = gpx_time_str.astype(str).str.rstrip('+00:00')
df_gpx['time'] = gpx_time_strip
df_gpx['time'] = pd.to_datetime(df_gpx['time'], format='ISO8601')
df_gpx['time'] = df_gpx['time'] + timedelta(hours=1)


print(df_gpx)
df_gpx.to_csv('test_gpx_output')

                    time   latitude  longitude  elevation
0    2024-05-24 20:28:53  51.427028  -0.042831  34.949332
1    2024-05-24 20:28:54  51.426989  -0.042799  35.057378
2    2024-05-24 20:28:55  51.427005  -0.042790  35.037367
3    2024-05-24 20:28:56  51.426987  -0.042757  35.016640
4    2024-05-24 20:28:57  51.427008  -0.042756  34.983199
...                  ...        ...        ...        ...
1410 2024-05-24 20:57:13  51.426937  -0.042565  33.525962
1411 2024-05-24 20:57:15  51.426962  -0.042581  33.546778
1412 2024-05-24 20:57:16  51.426973  -0.042595  33.286068
1413 2024-05-24 20:57:17  51.426977  -0.042604  33.205759
1414 2024-05-24 20:57:18  51.426986  -0.042620  33.137875

[1415 rows x 4 columns]


In [70]:
print(df['time'])
print(df_gpx['time'])

0   2024-05-24 20:28:01
0   2024-05-24 20:28:03
0   2024-05-24 20:28:03
0   2024-05-24 20:28:05
0   2024-05-24 20:28:05
            ...        
0   2024-05-24 20:57:34
0   2024-05-24 20:57:36
0   2024-05-24 20:57:36
0   2024-05-24 20:57:38
0   2024-05-24 20:57:41
Name: time, Length: 1120, dtype: datetime64[ns]
0      2024-05-24 20:28:53
1      2024-05-24 20:28:54
2      2024-05-24 20:28:55
3      2024-05-24 20:28:56
4      2024-05-24 20:28:57
               ...        
1410   2024-05-24 20:57:13
1411   2024-05-24 20:57:15
1412   2024-05-24 20:57:16
1413   2024-05-24 20:57:17
1414   2024-05-24 20:57:18
Name: time, Length: 1415, dtype: datetime64[ns]


In [93]:
#add dataframes from sensor and gpx into one file - match the time-steps
#if df_gpx['time'] = df['time']:
    
df_sens_gpx = pd.merge_ordered(df, df_gpx, fill_method="ffill", left_by='time')
lat_gpx = df_sens_gpx['latitude']
df_sens_gpx['latitude'] = lat_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')
long_gpx = df_sens_gpx['longitude']
df_sens_gpx['longitude'] = long_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')
ele_gpx = df_sens_gpx['elevation']
df_sens_gpx['elevation'] = ele_gpx.interpolate(method='linear', axis=0, limit=5, inplace=False, limit_area='inside')

print(df_sens_gpx)

df_sens_gpx.to_csv(encoded_sensor_and_gpx_path) #writes results file (with encoded sensor measurements)

                    time                         values  latitude  longitude  \
0    2024-05-24 20:28:01  E201 006B 0200 F102 00FE 0100       NaN        NaN   
1    2024-05-24 20:28:03            3212 088D 0100 D606       NaN        NaN   
2    2024-05-24 20:28:03  DF01 0082 0200 D202 00FA 0100       NaN        NaN   
3    2024-05-24 20:28:05            3312 FE8C 0100 D606       NaN        NaN   
4    2024-05-24 20:28:05  DB01 0062 0200 E002 00F6 0100       NaN        NaN   
...                  ...                            ...       ...        ...   
1183 2024-05-24 20:57:34  1102 00AD 0200 0203 002F 0200       NaN        NaN   
1184 2024-05-24 20:57:36            370E 088D 0100 9605       NaN        NaN   
1185 2024-05-24 20:57:36  0702 0097 0200 1403 0025 0200       NaN        NaN   
1186 2024-05-24 20:57:38  0002 0091 0200 0603 001E 0200       NaN        NaN   
1187 2024-05-24 20:57:41  F801 007F 0200 EB02 0015 0200       NaN        NaN   

      elevation  
0           NaN  
1  